In [35]:
from google.colab import drive
drive.mount('/content/drive')

drivePath = '/content/drive/MyDrive/Colab Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

Thu Jul  1 11:20:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip uninstall torch
!pip install torch

Uninstalling torch-1.8.1+cu111:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torch-1.8.1+cu111.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.8.1+cu111
     |████████████████████████████████| 831.4MB 17kB/s 
ERROR: Operation cancelled by user


In [4]:
SEED = 1234

import torch
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Download dataset from official website

In [5]:
#@title Download CoNSeP
!rm -rf CoNSeP
!wget https://warwick.ac.uk/fac/cross_fac/tia/data/hovernet/consep.zip
!unzip /content/consep.zip > /dev/null
!rm -rf _*; rm /content/consep.zip

--2021-07-01 11:35:57--  https://warwick.ac.uk/fac/cross_fac/tia/data/hovernet/consep.zip
Resolving warwick.ac.uk (warwick.ac.uk)... 137.205.28.41
Connecting to warwick.ac.uk (warwick.ac.uk)|137.205.28.41|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 153334740 (146M) [application/zip]
Saving to: ‘consep.zip’

consep.zip          100%[===================>] 146.23M  20.9MB/s    in 13s     

2021-07-01 11:36:11 (11.2 MB/s) - ‘consep.zip’ saved [153334740/153334740]



In [6]:
import os
import scipy.io
import numpy as np
from PIL import Image

In [7]:
#@title view labels and images
# mat = scipy.io.loadmat('/content/CoNSeP/Train/Labels/train_1.mat')

import matplotlib.pyplot as plt

# H = mat['inst_map']

# plt.imshow(H, interpolation='none')
# plt.show()

# H = mat['type_map']

# plt.imshow(H, interpolation='none')
# plt.show()

# image = np.array(Image.open('/content/CoNSeP/Train/Images/train_24.png'))
# H = image[:,:,3]

# plt.imshow(H, interpolation='none')
# plt.show()

# Data loaders

In [8]:
import torch.nn as nn

class ConsepSimplePadDataset(torch.utils.data.Dataset):
  def __init__(self, train = False, test = False):
        self.directory = 'CoNSeP/Train' if train else 'CoNSeP/Test'
        self.setname = 'train' if train else 'test'
        assert train != test

  def __len__(self):
        return len(os.listdir(os.path.join(self.directory, 'Images')))

  def __getitem__(self, index):
        # Load data and get label
        image = np.array(Image.open(os.path.join(self.directory, 'Images', self.setname + f'_{index + 1}.png')))[:,:,:3]
        image = torch.from_numpy(np.transpose(image, (2, 0, 1))) / 255

        labels = scipy.io.loadmat(os.path.join(self.directory, 'Labels', self.setname + f'_{index + 1}.mat'))
        label_inst = torch.from_numpy(labels['inst_map']).long()
        label_type = torch.from_numpy(labels['type_map']).long()

        m = nn.ZeroPad2d(12)

        return m(image), m(label_inst), m(label_type)

In [9]:
# CUDA for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 0}

# Generators
training_set = ConsepSimplePadDataset(train = True)
validation_len = int(len(training_set) * 0.3)

training_set, validation_set = torch.utils.data.random_split(training_set, [len(training_set) - validation_len, validation_len])
training_generator = torch.utils.data.DataLoader(training_set, **params)
validation_generator = torch.utils.data.DataLoader(validation_set, **params)

test_set = ConsepSimplePadDataset(test = True)
test_generator = torch.utils.data.DataLoader(test_set, **params)

In [10]:
n_type = 8
ratio_type = torch.ones(n_type)
ratio_inst = torch.ones(2)
for _, local_labels_inst, local_labels_type in training_generator:
    for i in range(n_type):
        ratio_type[i] += torch.sum(local_labels_type == i)
    ratio_inst[0] += torch.sum(local_labels_inst == 0)
    ratio_inst[1] += torch.sum(local_labels_inst > 0)

# Model

In [1]:
#@title clone UNetPlusPlus
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html >/dev/null
!pip install torchtext==0.9.1 >/dev/null
!pip install segmentation-models-pytorch >/dev/null
import segmentation_models_pytorch

tcmalloc: large alloc 1982177280 bytes == 0x55c085b26000 @  0x7f632be711e7 0x55c082e90e68 0x55c082e5b637 0x55c082fda605 0x55c082f740b8 0x55c082e5ef02 0x55c082f3cd1e 0x55c082e5eb59 0x55c082f4ffed 0x55c082ed2988 0x55c082e6030a 0x55c082ece60e 0x55c082e6030a 0x55c082ece60e 0x55c082ecd7ad 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd4ae 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd4ae 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd7ad 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd4ae
tcmalloc: large alloc 1982177280 bytes == 0x55c0fbd80000 @  0x7f632be711e7 0x55c082e90e68 0x55c082e5b637 0x55c082f717e7 0x55c082e5ec7c 0x55c082f4ffed 0x55c082ed2988 0x55c082e6030a 0x55c082ece60e 0x55c082e6030a 0x55c082ece60e 0x55c082ecd7ad 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd4ae 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd4ae 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ecd7ad 0x55c082e60c9f 0x55c082e60ea1 0x55c082ecfbb5 0x55c082ec

In [11]:
model = segmentation_models_pytorch.UnetPlusPlus(encoder_name='resnet34', encoder_depth=5, encoder_weights='imagenet', decoder_use_batchnorm=True, decoder_channels=(256, 128, 64, 32, 16), decoder_attention_type=None, in_channels=3, classes=n_type, activation=None, aux_params=None)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,079,624 trainable parameters


# Training

In [13]:
from torch import optim

optimizer = optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=1e-8, momentum=0.9)

if n_type > 1:
    criterion = nn.CrossEntropyLoss(weight = 1 / torch.log(ratio_type).to(device))
else:
    criterion = nn.CrossEntropyLoss(weight = 1 / torch.log(ratio_inst).to(device))

In [14]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).float().mean()
    acc = correct
    return acc

In [15]:
def binary_metric(preds, y, n_classes=8):
    rounded_preds = preds.argmax(1)
    scores = []
    t = rounded_preds == y
    for i in range(n_classes):
        p = rounded_preds == i
        tp = torch.sum(torch.logical_and(t, p).float(), dim = 1)
        fp = torch.sum(torch.logical_and(torch.logical_not(t), p).float(), dim = 1)
        fn = torch.sum(torch.logical_and(torch.logical_and(torch.logical_not(t), torch.logical_not(p)), y == i).float(), dim = 1)

        recall = tp / (tp + fn + 1e-10)
        prec = tp / (tp + fp + 1e-10)
        f1 = 2 * tp / (2 * tp + fp + fn + 1e-10)
        f2 = 5 * tp / (5 * tp + 4 * fn + fp + 1e-10)
        scores.append([recall.mean(), prec.mean(), f1.mean(), f2.mean()])
    
    return torch.tensor(scores)

In [16]:
from tqdm.notebook import tqdm

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    statistics = []

    model.train()

    for local_batch, local_labels_inst, local_labels_type in tqdm(iterator):

        # Transfer to GPU
        local_batch, local_labels_inst, local_labels_type = local_batch.to(device), local_labels_inst.to(device), local_labels_type.to(device)
        
        optimizer.zero_grad()
     
        type_pred = model(local_batch)
        
        loss = criterion(type_pred, local_labels_type)
        
        acc = categorical_accuracy(type_pred, local_labels_type)

        statistics.append(binary_metric(type_pred, local_labels_type))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), sum(statistics) / len(statistics)

In [17]:
def evaluate(model, iterator, criterion, demo = False):
    
    epoch_loss = 0
    epoch_acc = 0
    statistics = []

    model.eval()

    with torch.no_grad():

        for local_batch, local_labels_inst, local_labels_type in tqdm(iterator):

            # Transfer to GPU
            local_batch, local_labels_inst, local_labels_type = local_batch.to(device), local_labels_inst.to(device), local_labels_type.to(device)

            type_pred = model(local_batch)
        
            loss = criterion(type_pred, local_labels_type)
            
            acc = categorical_accuracy(type_pred, local_labels_type)

            statistics.append(binary_metric(type_pred, local_labels_type))

            epoch_loss += loss.item()
            epoch_acc += acc.item()

            #demo
            if demo:
                truth = local_labels_type.squeeze(0).cpu().numpy()
                plt.imshow(np.concatenate((type_pred.argmax(1).squeeze(0).cpu().numpy(), truth), axis = 1), interpolation='none')
                plt.show()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), sum(statistics) / len(statistics)

In [44]:
def quality_metric(preds, y, n_classes=8):
  
    rounded_preds = preds.argmax(1)
    scores = []
    t = rounded_preds == y
    for i in range(n_classes):
        p = rounded_preds == i
        l = y == i
        tp = torch.sum(torch.logical_and(t, p).float(), dim = 1)
        fp = torch.sum(torch.logical_and(torch.logical_not(t), p).float(), dim = 1)
        fn = torch.sum(torch.logical_and(torch.logical_and(torch.logical_not(t), torch.logical_not(p)), y == i).float(), dim = 1)
        
        f1 = 2 * tp / (2 * tp + fp + fn + 1e-10)

        intersection = (l & p).float().sum((1, 2)) 
        union = (l | p).float().sum((1, 2)) 
        
        iou = (intersection + 1e-10) / (union + 1e-10)

        scores.append([f1.mean(), iou, f1.mean()*iou])
    
    return torch.tensor(scores)

def AJI(preds, y, n_classes=8):
    rounded_preds = preds.argmax(1)
    total_intersection = 0
    total_union = 0
    for i in range(1, n_classes):
        p = rounded_preds == i
        l = y == i

        intersection = (l & p).float().sum((1, 2)) 
        union = (l | p).float().sum((1, 2)) 
        
        total_intersection += intersection
        total_union += union
    
    return (total_intersection + 1e-10) / (total_union + 1e-10)

def DICE_2(preds, y, n_classes=8):
    rounded_preds = preds.argmax(1)
    p = rounded_preds != 0
    l = y != 0
    total_intersection = 0
    for i in range(1, n_classes):
      m1 = rounded_preds == i
      m2 = y == i
      intersection = (m1 & m2).float().sum((1, 2))
      total_intersection += intersection 
    sumOfpl = l.float().sum((1, 2)) + p.float().sum((1, 2)) 
    DICE2 = 2* (total_intersection + 1e-10) / (sumOfpl + 1e-10)
    
    return DICE2

def classification_eval(preds, y):

    rounded_preds = preds.argmax(1)
    TPd = rounded_preds & y
    t = rounded_preds == y
    Ac = torch.logical_and(TPd, t).float().sum((1, 2))
    Bc = torch.logical_and(TPd, torch.logical_not(t)).float().sum((1, 2))
    incorrectDetection = torch.logical_xor(rounded_preds, y).float().sum((1, 2))
    
    Fc = (2*Ac+1e-10)/(2*(Ac+Bc)+incorrectDetection+1e-10)

    return Fc

In [41]:
def new_evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    # segmentation_scores = []
    DICE2_list = []
    AJI_list = []
    classification_scores = []
    model.eval()

    with torch.no_grad():

        for local_batch, local_labels_inst, local_labels_type in tqdm(iterator):

            # Transfer to GPU
            local_batch, local_labels_inst, local_labels_type = local_batch.to(device), local_labels_inst.to(device), local_labels_type.to(device)

            type_pred = model(local_batch)

            loss = criterion(type_pred, local_labels_type)
            
            acc = categorical_accuracy(type_pred, local_labels_type)

            epoch_loss += loss.item()
            epoch_acc += acc.item()


            DICE2_list.append(DICE_2(type_pred, local_labels_type))
            AJI_list.append(AJI(type_pred, local_labels_type))
            # segmentation_scores.append(quality_metric(type_pred, local_labels_type))
            classification_scores.append(classification_eval(type_pred, local_labels_type))

    return epoch_loss / len(iterator), epoch_acc / len(iterator), sum(DICE2_list) / len(DICE2_list), sum(AJI_list) / len(AJI_list), sum(classification_scores) / len(classification_scores)






In [42]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc, train_stats = train(model, training_generator, optimizer, criterion)
    valid_loss, valid_acc, valid_DICE2, valid_AJI, valid_classification_score = new_evaluate(model, validation_generator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), drivePath + 'unetplusplus.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('train metrics R, P, F1, F2')
    print((train_stats * 1e4).long())
    print('valid metrics DICE2, AJI')
    print((valid_DICE2 * 1e4).long(), (valid_AJI * 1e4).long())
    print('valid classification score: ', valid_classification_score)


Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 0.358 | Train Acc: 88.35%
	 Val. Loss: 0.372 |  Val. Acc: 90.79%
train metrics R, P, F1, F2
tensor([[9432, 9449, 9430, 9428],
        [   0,    0,    0,    0],
        [1341, 2115, 1468, 1361],
        [   0,    0,    0,    0],
        [3280, 3214, 3177, 3220],
        [2315, 2849, 2293, 2245],
        [1301, 1351, 1270, 1272],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2493], device='cuda:0') tensor([1648], device='cuda:0')
valid classification score:  tensor([0.2935], device='cuda:0')



Epoch: 02 | Epoch Time: 0m 19s
	Train Loss: 0.359 | Train Acc: 88.43%
	 Val. Loss: 0.351 |  Val. Acc: 88.92%
train metrics R, P, F1, F2
tensor([[9555, 9339, 9435, 9504],
        [   0,    0,    0,    0],
        [1496, 2114, 1542, 1478],
        [   0,    0,    0,    0],
        [3177, 3284, 3150, 3148],
        [1532, 2152, 1503, 1478],
        [1198, 1141, 1030, 1087],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2089], device='cuda:0') tensor([1274], device='cuda:0')
valid classification score:  tensor([0.2884], device='cuda:0')



Epoch: 03 | Epoch Time: 0m 19s
	Train Loss: 0.347 | Train Acc: 88.69%
	 Val. Loss: 0.383 |  Val. Acc: 89.10%
train metrics R, P, F1, F2
tensor([[9434, 9456, 9434, 9431],
        [   0,    0,    0,    0],
        [1489, 2168, 1597, 1497],
        [   0,    0,    0,    0],
        [3447, 3228, 3289, 3370],
        [2397, 3113, 2361, 2318],
        [1139, 1385, 1154, 1127],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2562], device='cuda:0') tensor([1653], device='cuda:0')
valid classification score:  tensor([0.3053], device='cuda:0')



Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 0.407 | Train Acc: 87.73%
	 Val. Loss: 0.745 |  Val. Acc: 77.11%
train metrics R, P, F1, F2
tensor([[9497, 9343, 9402, 9454],
        [   0,    0,    0,    0],
        [1313, 2000, 1384, 1310],
        [   0,    0,    0,    0],
        [3088, 3354, 3119, 3080],
        [1951, 2912, 2081, 1954],
        [1328,  847,  970, 1111],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([766], device='cuda:0') tensor([485], device='cuda:0')
valid classification score:  tensor([0.1032], device='cuda:0')



Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 0.358 | Train Acc: 88.48%
	 Val. Loss: 0.502 |  Val. Acc: 84.46%
train metrics R, P, F1, F2
tensor([[9403, 9480, 9431, 9412],
        [   0,    0,    0,    0],
        [1791, 2250, 1776, 1741],
        [   0,    0,    0,    0],
        [3453, 3129, 3212, 3333],
        [2257, 2746, 2212, 2178],
        [1064, 1220, 1034, 1035],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([340], device='cuda:0') tensor([174], device='cuda:0')
valid classification score:  tensor([0.0602], device='cuda:0')


...


Epoch: 46 | Epoch Time: 0m 19s
	Train Loss: 0.260 | Train Acc: 90.89%
	 Val. Loss: 0.337 |  Val. Acc: 90.59%
train metrics R, P, F1, F2
tensor([[9517, 9549, 9526, 9519],
        [ 131,  315,  168,  142],
        [2300, 2510, 2268, 2254],
        [   0,    0,    0,    0],
        [3605, 3487, 3505, 3554],
        [3675, 3944, 3588, 3588],
        [1531, 1397, 1393, 1451],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2383], device='cuda:0') tensor([1555], device='cuda:0')
valid classification score:  tensor([0.2989], device='cuda:0')



Epoch: 47 | Epoch Time: 0m 19s
	Train Loss: 0.253 | Train Acc: 91.09%
	 Val. Loss: 0.366 |  Val. Acc: 89.99%
train metrics R, P, F1, F2
tensor([[9504, 9578, 9535, 9515],
        [ 220,  363,  247,  227],
        [2294, 2347, 2191, 2217],
        [   0,    0,    0,    0],
        [3631, 3510, 3531, 3580],
        [3665, 3874, 3541, 3563],
        [1473, 1497, 1449, 1454],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2140], device='cuda:0') tensor([1399], device='cuda:0')
valid classification score:  tensor([0.2758], device='cuda:0')



Epoch: 48 | Epoch Time: 0m 19s
	Train Loss: 0.240 | Train Acc: 91.44%
	 Val. Loss: 0.373 |  Val. Acc: 91.44%
train metrics R, P, F1, F2
tensor([[9576, 9538, 9552, 9566],
        [ 393,  565,  412,  394],
        [2649, 2550, 2485, 2549],
        [   0,    0,    0,    0],
        [3570, 3570, 3538, 3549],
        [3368, 4108, 3471, 3366],
        [1656, 1416, 1490, 1572],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2574], device='cuda:0') tensor([1710], device='cuda:0')
valid classification score:  tensor([0.3054], device='cuda:0')



Epoch: 49 | Epoch Time: 0m 19s
	Train Loss: 0.249 | Train Acc: 91.35%
	 Val. Loss: 0.412 |  Val. Acc: 91.64%
train metrics R, P, F1, F2
tensor([[9534, 9556, 9539, 9534],
        [ 432,  601,  460,  437],
        [2324, 2511, 2302, 2290],
        [   0,    0,    0,    0],
        [3643, 3519, 3545, 3594],
        [3669, 3973, 3579, 3582],
        [1533, 1445, 1425, 1474],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2678], device='cuda:0') tensor([1795], device='cuda:0')
valid classification score:  tensor([0.3150], device='cuda:0')



Epoch: 50 | Epoch Time: 0m 19s
	Train Loss: 0.271 | Train Acc: 91.07%
	 Val. Loss: 0.387 |  Val. Acc: 91.47%
train metrics R, P, F1, F2
tensor([[9545, 9528, 9529, 9537],
        [ 184,  379,  222,  195],
        [2319, 2525, 2281, 2274],
        [   0,    0,    0,    0],
        [3588, 3507, 3500, 3541],
        [3341, 3941, 3420, 3335],
        [1525, 1363, 1391, 1452],
        [   0,    0,    0,    0]])
valid metrics DICE2, AJI
tensor([2206], device='cuda:0') tensor([1433], device='cuda:0')
valid classification score:  tensor([0.2719], device='cuda:0')


In [ ]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc, train_stats = train(model, training_generator, optimizer, criterion)
    valid_loss, valid_acc, valid_stats = evaluate(model, validation_generator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), drivePath + 'unetplusplus.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('train metrics R, P, F1, F2')
    print((train_stats * 1e4).long())
    print('valid metrics R, P, F1, F2')
    print((valid_stats * 1e4).long())

In [ ]:
model.load_state_dict(torch.load(drivePath + 'unetplusplus.pt'))

test_loss, test_acc, test_stats = evaluate(model, test_generator, criterion, demo = True)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
print('test metrics R, P, F1, F2')
print((test_stats * 1e4).long())